In [ ]:
import pandas as pd
import utils 
import plotly.graph_objects as go 
import instrument

In [ ]:
pair = "CAD_CHF"
granularity = "H1"
ma_list=[16,64]
i_pair = instrument.Instrument.get_instrument_by_name(pair)

In [ ]:
df = pd.read_pickle(utils.get_his_data_filename(pair, granularity))
non_cols = ["time", "volume"]
mod_cols = [x for x in df.columns if x not in non_cols]
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [ ]:
df_ma = df[["time", "mid_o", "mid_h", "mid_l", "mid_c"]].copy()
for ma in ma_list:
    df_ma[f'MA_{ma}'] = df.mid_c.rolling(window=ma).mean()
df_ma.dropna(inplace=True)
df_ma.reset_index(drop=True, inplace=True)

In [ ]:
df_ma.head()

In [ ]:
def is_trade(row):
    if row.DIFF >= 0 and row.DIFF_PREV < 0:
        return 1 
    if row.DIFF <= 0 and row.DIFF_PREV > 0:
        return -1 
    return 0

In [ ]:
df_ma['DIFF'] = df_ma.MA_16 - df_ma.MA_64
df_ma['DIFF_PREV'] = df_ma.DIFF.shift(1)
df_ma['IS_TRADE'] = df_ma.apply(is_trade, axis=1)
df_trades = df_ma[df_ma.IS_TRADE!=0].copy()

In [ ]:
df_trades.head()

In [ ]:
df_trades["DELTA"] = (df_trades.mid_c.diff() / i_pair.pipLocation).shift(-1)
df_trades["GAIN"] = df_trades["DELTA"] * df_trades["IS_TRADE"]

In [ ]:
df_trades["GAIN"].sum()

In [ ]:
from dateutil.parser import *
df_trades["time"] = [parse(x) for x in df_trades.time]

In [ ]:
df_trades["DURATION"] = df_trades.time.diff().shift(-1)

In [ ]:
df_trades.info()

In [51]:
df_all = pd.read_pickle("all_trades.pkl")

In [52]:
df_all.describe()

,mid_c,DIFF,DIFF_PREV,IS_TRADE,DELTA,GAIN,MALONG,MASHORT,DURATION
count,66470.000000,6.647000e+04,6.647000e+04,66470.000000,66470.000000,66470.000000,66470.000000,66470.000000,66470.000000
mean,36.816329,-5.257852e-04,5.504369e-04,-0.002106,2.996401,-7.122518,12.127095,63.184835,50.326583
std,59.196538,3.382478e-02,3.239599e-02,1.000005,84.090244,83.841616,12.829569,65.661671,79.869579
min,0.544740,-8.646146e-01,-6.562969e-01,-1.000000,-812.900000,-606.300000,4.000000,8.000000,1.000000
25%,0.916490,-2.543750e-04,-2.463281e-04,-1.000000,-27.000000,-39.200000,4.000000,16.000000,6.000000
50%,1.347880,-2.604167e-07,6.250000e-07,-1.000000,1.700000,-13.200000,8.000000,32.000000,19.000000
75%,84.296000,2.513932e-04,2.512500e-04,1.000000,32.000000,10.400000,16.000000,96.000000,67.000000
max,172.111000,6.221250e-01,1.131125e+00,1.000000,1327.900000,1327.900000,64.000000,256.000000,1516.000000


In [ ]:
df_plot = df_ma.iloc[:100].copy()

In [ ]:
df_plot.shape

In [ ]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor='#24A06B',
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color='#2EC886',  
    decreasing_line_color='#FF3A4C'
))
for ma in ma_list:  
    col = f"MA_{ma}"
    fig.add_trace(go.Scatter(x=df_plot.time, 
        y=df_plot[col],
        line=dict(width=2),
        line_shape='spline',
        name=col
        ))
fig.update_layout(width=1000,height=400,
    margin=dict(l=10,r=10,b=10,t=10),
    font=dict(size=10,color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True
)
fig.show()